In [378]:
import re
import numpy as np
from scipy.ndimage import convolve

In [379]:
with open('day24_input.txt', 'r') as f:
    data = f.read().splitlines()
dirs = re.compile('e|se|sw|w|nw|ne')
data = [re.findall(dirs, line) for line in data]

In [380]:
def get_position(tile):
    pos = {'x': 0, 'y': 0}
    for step in tile:
        if step == 'e':
            pos['x'] += 1
        elif step == 'w':
            pos['x'] -= 1
        elif step == 'ne':
            pos['x'] += 0.5
            pos['y'] += 1
        elif step == 'sw':
            pos['x'] -= 0.5
            pos['y'] -= 1
        elif step == 'se':
            pos['x'] += 0.5
            pos['y'] -= 1
        elif step == 'nw':
            pos['x'] -= 0.5
            pos['y'] += 1
    return pos

In [381]:
test_data = """sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew""".splitlines()
test_data = [re.findall(dirs, line) for line in test_data]

In [382]:
black_tiles = set()
for tile in data:
    tile_pos = tuple(get_position(tile).values())
    if tile_pos in black_tiles:
        black_tiles.remove(tile_pos)
    else:
        black_tiles.add(tile_pos)

In [383]:
len(black_tiles)

382

In [384]:
# Map hexagonal grid to 2D coords
def get_position_cube(tile):
    pos = {'r': 0, 'q': 0}
    for step in tile:
        if step == 'e':
            pos['q'] += 1
        elif step == 'w':
            pos['q'] -= 1
        elif step == 'ne':
            pos['q'] += 1
            pos['r'] -= 1
        elif step == 'sw':
            pos['q'] -= 1
            pos['r'] += 1
        elif step == 'se':
            pos['r'] += 1
        elif step == 'nw':
            pos['r'] -= 1
    return pos

In [387]:
black_tiles = set()
for tile in data:
    tile_pos = tuple(get_position_cube(tile).values())
    if tile_pos in black_tiles:
        black_tiles.remove(tile_pos)
    else:
        black_tiles.add(tile_pos)

In [388]:
black_idxs = np.array(list(black_tiles))
black_idxs += np.ptp(black_idxs, 0)//2

In [390]:
mask = np.ones([3, 3]) - np.diag([1, 1, 1])

board = np.zeros(np.ptp(black_idxs, 0) + 1)
board[black_idxs[:, 0], black_idxs[:, 1]] = 1

for i in range(100):
    board = np.pad(board, [(1,1), (1,1)], mode='constant', constant_values=0)
    conv_board = convolve(board, mask, mode='constant', cval=0)
    board = (((board == 1) != ((board == 1) & ((conv_board == 0) | (conv_board > 2)))) | 
             ((board == 0) & (conv_board == 2))).astype(np.uint8)

In [391]:
board.sum()

3964